<a href="https://colab.research.google.com/github/ericyoc/bgp-routing-attacks-sim-poc/blob/main/bgp_routing_attacks_sim_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
from collections import defaultdict
from tabulate import tabulate
from textwrap import wrap

class Router:
    def __init__(self, asn):
        self.asn = asn
        self.neighbors = []
        self.routing_table = defaultdict(lambda: float('inf'))
        self.routing_table[asn] = (0, asn, [asn])

    def add_neighbor(self, neighbor):
        self.neighbors.append(neighbor)

    def receive_update(self, asn, distance, next_hop, path):
        if asn not in self.routing_table or distance + 1 < self.routing_table[asn][0]:
            self.routing_table[asn] = (distance + 1, next_hop, path + [self.asn])

    def send_updates(self):
        for neighbor in self.neighbors:
            for asn, (distance, next_hop, path) in self.routing_table.items():
                if asn != neighbor.asn:
                    neighbor.receive_update(asn, distance, self.asn, path)

    def get_best_path(self, asn):
        distance, next_hop, path = self.routing_table[asn]
        if distance == float('inf'):
            return None, None, None
        return distance, next_hop, path

def simulate_bgp(routers, iterations):
    for _ in range(iterations):
        for router in routers:
            router.send_updates()

def print_routing_table(routers, attacker_asn, target_asn, explanation, routing_paths):
    table_data = []
    headers = ["Router"] + [f"AS{router.asn}" for router in routers]

    table_data.append([f"Attacker ASN: {attacker_asn}", "", "", "", ""])
    table_data.append([f"Target ASN: {target_asn}", "", "", "", ""])
    table_data.append([""] * len(headers))

    for router in routers:
        row_data = [f"Router {router.asn}"]
        for asn in range(1, len(routers) + 1):
            distance, next_hop, path = router.get_best_path(asn)
            if distance is None:
                row_data.append("∞")
            else:
                row_data.append(f"{distance} (AS{next_hop}) {path}")
        table_data.append(row_data)

    print(tabulate(table_data, headers, tablefmt="grid"))

    wrapped_explanation = wrap(explanation, width=80)
    print("\nExplanation:")
    for line in wrapped_explanation:
        print(line)

    print("\nRouting Paths:")
    for path in routing_paths:
        print(path)

def perform_subprefix_hijacking(routers, attacker_asn, target_asn):
    attacker_router = None
    for router in routers:
        if router.asn == attacker_asn:
            attacker_router = router
            break

    if attacker_router:
        # Perform sub-prefix hijacking by advertising a more specific prefix of the target ASN
        attacker_router.routing_table[target_asn] = (0, attacker_asn, [attacker_asn])
        attacker_router.send_updates()

def perform_bgp_prepending(routers, prepender_asn, prepend_count):
    prepender_router = None
    for router in routers:
        if router.asn == prepender_asn:
            prepender_router = router
            break

    if prepender_router:
        # Perform BGP prepending by adding multiple instances of the prepender's ASN to the path
        for asn, (distance, next_hop, path) in prepender_router.routing_table.items():
            prepender_router.routing_table[asn] = (distance, next_hop, [prepender_asn] * prepend_count + path)
        prepender_router.send_updates()

def get_routing_paths(routers, target_asn):
    routing_paths = []
    for router in routers:
        distance, next_hop, path = router.get_best_path(target_asn)
        routing_paths.append(" -> ".join(f"AS{asn}" for asn in path))
    return routing_paths

def main():
    # Create routers
    routers = [Router(1), Router(2), Router(3), Router(4)]

    # Set up neighbor relationships
    routers[0].add_neighbor(routers[1])
    routers[0].add_neighbor(routers[2])
    routers[1].add_neighbor(routers[0])
    routers[1].add_neighbor(routers[3])
    routers[2].add_neighbor(routers[0])
    routers[2].add_neighbor(routers[3])
    routers[3].add_neighbor(routers[1])
    routers[3].add_neighbor(routers[2])

    # Define the attacker and target ASNs
    attacker_asn = 4
    target_asn = 1

    # Simulate BGP routing before the attack
    print("Routing Table before BGP Attacks:")
    before_explanation = "Before the BGP attacks, the routing table shows the normal distances, next-hop ASNs, and paths between routers. Each router has learned the best paths to reach other ASNs based on the shortest distances advertised by their neighbors."
    simulate_bgp(routers, 5)
    before_routing_paths = get_routing_paths(routers, target_asn)
    print_routing_table(routers, attacker_asn, target_asn, before_explanation, before_routing_paths)

    # Perform sub-prefix hijacking
    print("\nRouting Table after Sub-Prefix Hijacking:")
    subprefix_explanation = "In a sub-prefix hijacking attack, the attacker (AS4) advertises a more specific prefix of the target ASN (AS1). This causes other routers to prefer the more specific route advertised by the attacker, effectively hijacking the traffic destined for the target ASN."
    perform_subprefix_hijacking(routers, attacker_asn, target_asn)
    simulate_bgp(routers, 5)
    subprefix_routing_paths = get_routing_paths(routers, target_asn)
    print_routing_table(routers, attacker_asn, target_asn, subprefix_explanation, subprefix_routing_paths)

    # Perform BGP prepending
    prepender_asn = 2
    prepend_count = 3
    print("\nRouting Table after BGP Prepending:")
    prepending_explanation = f"In a BGP prepending attack, the prepender (AS{prepender_asn}) artificially increases the length of the AS path by prepending its own ASN multiple times ({prepend_count}). This makes the route appear less attractive to other routers, discouraging them from using the prepender as a transit point."
    perform_bgp_prepending(routers, prepender_asn, prepend_count)
    simulate_bgp(routers, 5)
    prepending_routing_paths = get_routing_paths(routers, target_asn)
    print_routing_table(routers, attacker_asn, target_asn, prepending_explanation, prepending_routing_paths)

if __name__ == "__main__":
    main()

Routing Table before BGP Attacks:
+-----------------+-------------------+-------------------+-------------------+-------------------+
| Router          | AS1               | AS2               | AS3               | AS4               |
+=================+===================+===================+===================+===================+
| Attacker ASN: 4 |                   |                   |                   |                   |
+-----------------+-------------------+-------------------+-------------------+-------------------+
| Target ASN: 1   |                   |                   |                   |                   |
+-----------------+-------------------+-------------------+-------------------+-------------------+
|                 |                   |                   |                   |                   |
+-----------------+-------------------+-------------------+-------------------+-------------------+
| Router 1        | 0 (AS1) [1]       | 1 (AS2) [2, 1]    | 1 (AS3